## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

In [ ]:
%pip install okareo
%pip install chromadb
%pip install pandas

In [ ]:
# Webbizz is an example web business. 
# We load 10 short summaries about different business aspects to the vector database.
import chromadb
import pandas as pd    

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="retrieval_test", metadata={"hnsw:space": "cosine"})
jsonObj = pd.read_json(path_or_buf='./webbizz_10_articles.jsonl', lines=True)

collection.add(
    documents=list(jsonObj.input),
    ids=list(jsonObj.result)
)   

In [3]:
# We have a set of 20 questions, each asking about different business aspects. 
# There is a specific summary that is the target answer for the question, which is linked by UUID.
# We upload these questions into a scenario set, so that they can be used in a test run in the next cell
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

scenario = okareo.upload_scenario_set(file_path='./webbizz_retrieval_questions.jsonl', scenario_name="Retrieval Articles Scenario")
print(scenario) # use scenario set id in output for the next step

ScenarioSetResponse(scenario_id='33d44f39-b929-43d8-8972-1ca5cfc1c0e1', project_id='4d3d47fb-1836-409f-af1f-3dc5182208fe', time_created=datetime.datetime(2024, 1, 18, 2, 31, 52, 564472), type='SEED', tags=None, name='Retrieval Articles Scenario', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})


In [6]:
# Perform a test run using a scenario set loaded in the previous cell 
from datetime import datetime
from okareo_api_client.models import TestRunType
from okareo.model_under_test import CustomModel

def query_results_to_score(results):
    parsed_ids_with_scores = []
    for i in range(0, len(results['distances'][0])):
        score = (2 - results['distances'][0][i]) / 2 # chromadb specific handling, this turns cosine distance into a 0 to 1 cosine similarity score
        parsed_ids_with_scores.append((results['ids'][0][i], score))
    return parsed_ids_with_scores
     

class RetrievalModel(CustomModel):
    def invoke(self, input: str):
        results = collection.query(
            query_texts=[input],
            n_results=5
        )
        # return a tuple of (parsed_ids_with_scores, overall model response context)
        return query_results_to_score(results), {'model_data': input}

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="vectordb_retrieval_test", model=RetrievalModel(name="custom retrieval"))

test_run_item = model_under_test.run_test(scenario=scenario, # use the scenario from the scenario set uploaded in the previous step
                                          name=f"Retrieval Test Run {datetime.now().strftime('%m-%d %H:%M:%S')}", # name for test run
                                          test_run_type=TestRunType.INFORMATION_RETRIEVAL,
                                          calculate_metrics=True)

# display model level metrics for the test run
print(f"See test run results: https://app.okareo.com/project/{test_run_item.project_id}/eval/{test_run_item.id}")

See test run results: https://app.okareo.com/project/4d3d47fb-1836-409f-af1f-3dc5182208fe/eval/815c5309-e984-4f49-bf21-750353451425
